# Create a catalog order

- <a href="#step-1">Step 1. Set up the notebook</a>
- <a href="#step-2">Step 2. Get access</a>
- <a href="#step-3">Step 3. Find and preview data</a>
- <a href="#step-4">Step 4. Create an order</a>
- <a href="#step-5">Step 5. Monitor orders</a>
- <a href="#step-6">Step 6. Download assets</a>

## <a id="step-1"></a> Step 1. Set up the notebook

### 1. Install prerequisites

In [1]:
!pip install up42-py --upgrade -q

import up42, rasterio

### 2. Authenticate

Use your credentials to authenticate.

In [2]:
# Create a configuration file and use the following code for it:
# {
#   "username": "<your-email-address>",
#   "password": "<your-password>"
# }

up42.authenticate(cfg_file="/Users/max.mustermann/.up42/credentials.json")

2024-04-10 14:38:47,262 - Got credentials from config file.
2024-04-10 14:38:47,809 - Authentication with UP42 successful!


## <a id="step-2"></a> Step 2. Get access

### 1. Select a catalog collection

Choose a catalog collection and get its `data_product_id` for ordering.

In [3]:
glossary = up42.glossary.ProductGlossary
catalog_type = up42.glossary.CollectionType.ARCHIVE
sort_by = up42.glossary.CollectionSorting.name
data_collections = glossary.get_collections(collection_type=catalog_type, sort_by=sort_by)
for collection in data_collections:
    print(f"collection name: {collection.name}")
    data_products = collection.data_products
    for data_product in data_products:
        print (f"data product name: {data_product.name}, id: {data_product.id}")

Choose a data product and copy its ID.

In [4]:
# The following value is the product id for Pléiades catalog — Analytic:
data_product_id = "4f1b2f62-98df-4c74-81f4-5dce45deee99"

### 2. Request access

If you want to order the chosen collection for the first time, you need to [request access to it](https://docs.up42.com/getting-started/restrictions#request-access).

An email from the Customer Success team usually takes up to 3 days. You can review your access request status on the [Access requests](https://console.up42.com/settings/access) page.

### 3. Accept the EULA

If you want to order the chosen collection for the first time, you need to accept its [end-user license agreement (EULA)](https://docs.up42.com/getting-started/account/eulas).

## <a id="step-3"></a> Step 3. Find and preview data

### 1. Search the catalog

Specify search parameters to find a full scene that fits your requirements.

In [5]:
catalog = up42.initialize_catalog()

geometry = up42.get_example_aoi()

search_parameters = catalog.construct_search_parameters(
    collections=["phr"],
    geometry=geometry,
    start_date="2022-06-01",
    end_date="2022-12-31",
    max_cloudcover=20,
    limit=10,
)

search_results_df = catalog.search(search_parameters)

2024-04-08 15:37:49,320 - Getting small example AOI in location 'Berlin'.
2024-04-08 15:37:49,497 - Searching catalog with search_parameters: {'datetime': '2022-06-01T00:00:00Z/2022-12-31T23:59:59Z', 'intersects': {'type': 'Polygon', 'coordinates': (((13.375777664180191, 52.51799543512652), (13.375777664180191, 52.514561554285706), (13.381566455794598, 52.514561554285706), (13.381566455794598, 52.51799543512652), (13.375777664180191, 52.51799543512652)),)}, 'limit': 10, 'collections': ['phr'], 'query': {'cloudCoverage': {'lte': 20}}}
2024-04-08 15:37:51,269 - 5 results returned.


Output a dataframe with full scenes that match the specified parameters.

In [6]:
search_results_df
# catalog.search(as_dataframe=False) # Uncomment to output JSON instead

### 2. Preview data

Some data hosts provide free, low-resolution quicklooks that you can preview before creating an order. Note that some quicklooks may not fully match the scene they represent.

In [7]:
# Download quicklooks
result = search_results_df.iloc[0]
catalog.download_quicklooks(
    image_ids=[result.sceneId],
    collection=result.constellation,
    output_directory="./quicklooks/",
)

# Define a function to georeference quicklooks
def georeference_quicklooks(
    src_path: str, dst_path: str, ulx: float, uly: float, lrx: float, lry: float
):
    with rasterio.open(src_path) as src:
        data = src.read()
        transform = rasterio.transform.from_bounds(
            ulx, lry, lrx, uly, data.shape[2], data.shape[1]
        )

        with rasterio.open(
            dst_path,
            "w",
            driver=src.driver,
            height=src.height,
            width=src.width,
            count=src.count,
            dtype=src.dtypes[0],
            crs="EPSG:4326",
            transform=transform,
        ) as dst:
            dst.write(data)

# Georefence the quicklooks
src_path = f"./quicklooks/quicklook_{result.sceneId}.jpg"
out_path = f"./quicklooks/georeferenced_quicklook_{result.sceneId}.png"
georeference_quicklooks(
    src_path,
    out_path,
    result.geometry.bounds[0],
    result.geometry.bounds[3],
    result.geometry.bounds[2],
    result.geometry.bounds[1],
)

2024-04-08 15:37:55,530 - Downloading quicklooks from provider oneatlas.
2024-04-08 15:37:55,531 - Download directory: quicklooks
100%|██████████| 1/1 [00:01<00:00,  1.37s/it]
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


Open the `quicklooks` folder. You will find the following files added by the cell above:

- An original quicklook
- A metadata file needed for georeferencing
- A georeferenced quicklook with the `georeferenced` prefix

[Visualize](https://docs.up42.com/help/visualize) the georeferenced quicklook in a third-party GIS software.

## <a id="step-4"></a> Step 4. Create an order

### 1. Fill out an order form

Fill out the order form for the chosen data product.

In [8]:
order_parameters = catalog.construct_order_parameters(
    data_product_id=data_product_id,
    image_id=search_results_df.iloc[0]["id"], # Note that the count starts from 0
    aoi=geometry,
)

2024-04-08 15:38:55,888 - See `catalog.get_data_product_schema(data_product_id)` for more detail on the parameter options.


Check the parameters by printing them.

In [9]:
order_parameters

{'dataProduct': '4f1b2f62-98df-4c74-81f4-5dce45deee99',
 'params': {'id': 'a4c9e729-1b62-43be-82e4-4e02c31963dd',
  'aoi': {'type': 'Polygon',
   'coordinates': (((13.375777664180191, 52.51799543512652),
     (13.375777664180191, 52.514561554285706),
     (13.381566455794598, 52.514561554285706),
     (13.381566455794598, 52.51799543512652),
     (13.375777664180191, 52.51799543512652)),)}}}

### 2. Get a cost estimate

Get a cost estimation before placing a catalog order.

In [10]:
catalog.estimate_order(order_parameters)

2024-04-08 15:39:06,345 - Order is estimated to cost 150 UP42 credits (order_parameters: {'dataProduct': '4f1b2f62-98df-4c74-81f4-5dce45deee99', 'params': {'id': 'a4c9e729-1b62-43be-82e4-4e02c31963dd', 'aoi': {'type': 'Polygon', 'coordinates': (((13.375777664180191, 52.51799543512652), (13.375777664180191, 52.514561554285706), (13.381566455794598, 52.514561554285706), (13.381566455794598, 52.51799543512652), (13.375777664180191, 52.51799543512652)),)}}})


150

The response returns the overall credit amount that will be deducted from your credit balance if you decide to proceed with the ordering.

### 3. Create an order

Create an order with the requested parameters.

Credits will be deducted upon successful completion of the created catalog order. The transaction can't be reversed.

In [11]:
order = catalog.place_order(order_parameters)
order

2024-04-08 15:39:13,206 - Order is PLACED
2024-04-08 15:39:13,207 - Order 34bc3737-71e4-46d0-88d5-92a9d82aab0a is now PLACED.


Order(order_id: 34bc3737-71e4-46d0-88d5-92a9d82aab0a, assets: [], dataProvider: None, status: PLACED, createdAt: 2024-04-08T13:39:10.434953Z, updatedAt: 2024-04-08T13:39:12.785075Z)

## <a id="step-5"></a> Step 5. Monitor orders

Check the [status](https://docs.up42.com/developers/api-catalog#order-statuses) of your order. To learn about the timeframe of catalog orders, see [Asset delivery time](https://docs.up42.com/data/archive/delivery).

In [12]:
order.status

2024-04-08 15:39:16,922 - Order is PLACED


'PLACED'

You can also track the order status until the order is completed.

In [13]:
order.track_status()

2024-04-08 15:39:19,209 - Tracking order status, reporting every 120 seconds...
2024-04-08 15:39:19,314 - Order is PLACED
2024-04-08 15:39:19,431 - Order is PLACED


## <a id="step-6"></a> Step 6. Download assets

The following statuses mean you can download assets from storage:
- `BEING_FULFILLED`: Some order assets might have been delivered. [Download](https://github.com/up42/documentation-notebooks/blob/main/storage/sdk-storage-assets.ipynb) these assets using the `Storage` class.
- `FULFILLED`: All order assets have been delivered. [Download](https://github.com/up42/documentation-notebooks/blob/main/storage/sdk-order-assets.ipynb) these assets using the `Order` class.